# Model H0

Model chứa các phương thức chung nhất mà mọi model tiếp theo phải kế thừa

In [ ]:
class H0(torch.nn.Module):
    def __init__(self):

    def save_model(self, folder_path, file_name, type='.pt'):

    def load_model(self, file_path, device='cpu'):

    def get_feature_maps(self, x):

    def visualize_feature_maps(self, input_image, show_last_layer=False):

# Model H1

Đây là model sử dụng CNN đơn giản

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from H0 import H0


class H1(H0):
    def __init__(self):
        super(H1, self).__init__()
        # Định nghĩa các tầng CNN
        # Tầng 1: Convolutional layer với 3 input channels (ảnh RGB), 32 output channels, kernel size 3
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # Batch Normalization sau tầng conv1
        # Tầng 2: Convolutional layer với 32 input channels, 64 output channels, kernel size 3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # Batch Normalization sau tầng conv2
        # Tầng 3: Convolutional layer với 64 input channels, 128 output channels, kernel size 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # Batch Normalization sau tầng conv3
        # Pooling layer giảm kích thước bằng cách lấy max trong cửa sổ 2x2
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.5)  # Dropout để giảm overfitting sau pooling
        # Fully connected layer để phân loại, 128 * 28 * 28 là số đặc trưng đầu vào từ tầng cuối cùng của CNN,
        # giả sử sau 3 lần pooling kích thước ảnh giảm xuống còn 28x28
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.dropout2 = nn.Dropout(0.6)  # Dropout trước tầng output để giảm overfitting
        # Output layer với 3 units cho 3 lớp phân loại
        self.fc2 = nn.Linear(512, 3)

    def forward(self, x):
        # Áp dụng các tầng convolutional và non-linearity (ReLU)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout1(x)  # Sử dụng lại dropout1
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.dropout1(x)  # Sử dụng lại dropout1
        # Flatten đầu ra trước khi đưa vào tầng fully connected
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Model H2

Sử dụng kiến trúc tương tự H1 nhưng thêm lớp Convolution đặt ngay đầu (trước khi truyền sang H1) là không train được, lớp Convolution này có nhiệm vụ làm nổi bật các góc cạnh có trong hình ảnh. Khi train thực tế sẽ đặt Convolution đầu tiên không train được, trọng số của các lớp còn lại để như của model H1 rồi thực hiện việc train.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from H0 import H0
from H1 import H1


class H2(H0):
    def __init__(self):
        super(H2, self).__init__()
        sobel_filter_vertical = torch.tensor(
            [[[-1.0, 0.0, 1.0], [-2.0, 0.0, 2.0], [-1.0, 0.0, 1.0]]],
            dtype=torch.float32
        ).repeat(3, 3, 1, 1)  
        sobel_filter_horizontal = torch.tensor(
            [[[1.0, 2.0, 1.0], [0.0, 0.0, 0.0], [-1.0, -2.0, -1.0]]],
            dtype=torch.float32
        ).repeat(3, 3, 1, 1)  
        sobel_filter = sobel_filter_vertical + sobel_filter_horizontal
        ## Khởi tạo lớp Conv2d
        self.conv0 = nn.Conv2d(3, 3, kernel_size=3, padding=1, bias=False)
        ## Đặt trọng số cho lớp Conv2d
        self.conv0.weight.data = sobel_filter
        ## Lớp Convolution đầu tiên không được train
        self.conv0.weight.requires_grad = False
        self.H1 = H1()

    def forward(self, x):
        x = self.conv0(x)
        x = self.H1(x)
        return x

    def get_feature_maps(self, x):

# Model H3

Đây là model có kết hợp sử dụng ResNet50 thay cho các lớp CNN đơn giản

Sau đây là hình ảnh cho kiến trúc của ResNet50 tham khảo của [Mahmood el al (2020). Automatic Hierarchical Classification of Kelps Using Deep Residual Features. Sensors. 20. 447](https://www.mdpi.com/1424-8220/20/2/447)

![img](img/ResNet50_Architecture.png)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from H0 import H0

class H3(H0):
    def __init__(self):
        super(H3, self).__init__()
        # Tải mô hình ResNet50 đã được huấn luyện trước
        resnet50 = models.resnet50(pretrained=True)
        # Loại bỏ lớp fully connected cuối cùng
        self.feature_extractor = nn.Sequential(*list(resnet50.children())[:-1])
        # Đóng băng các tham số trong feature extractor để không cập nhật trong quá trình huấn luyện
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        # Kích thước đầu vào cho lớp fully connected đầu tiên dựa trên output của ResNet50
        # ResNet50 thường trả về tensor [batch_size, 2048, 1, 1] sau lớp pooling cuối cùng
        self.fc1 = nn.Linear(2048, 512)
        self.dropout1 = nn.Dropout(0.5)
        # Output layer với 3 units cho 3 lớp phân loại
        self.fc2 = nn.Linear(512, 3)

    def forward(self, x):
        # Trích xuất đặc trưng
        x = self.feature_extractor(x)
        # Chuyển đổi tensor từ [batch_size, 2048, 1, 1] sang [batch_size, 2048] để phù hợp với lớp fully connected
        x = torch.flatten(x, 1)
        # Đưa qua mạng dense
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        return x
        
    def get_feature_maps(self, x):

# Model H4

Model kết hợp các lớp CNN với ViT:
- CNN trước ViT sau
- ViT trước CNN sau
- Cả CNN và ViT cùng lúc